In [2]:
import os
%pwd

'g:\\PROJECT\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'g:\\PROJECT\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [9]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories




In [10]:
class ConfigurationManager:
    def __init__(self,
                config_path=CONFIG_FILE_PATH,
                params_path=PARAMS_FILE_PATH):
        self.config= read_yaml(config_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config
        

In [12]:
import os
from src.TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

## DATA TRNASFORMATION COMP0ONENT

In [35]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_name)


    def cvrt_examples_to_features(self,ex_batch):
        
        input_encodings=self.tokenizer(ex_batch['dialogue'], max_length=511, truncation=True)
        with self.tokenizer.as_target_tokenizer():
         target_encodings=self.tokenizer(ex_batch['summary'], max_length=180, truncation=True)

        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask' :input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']

            }

    def convert(self):
        ds=load_from_disk(self.config.data_path)
        ds1=ds.map(self.cvrt_examples_to_features, batched=True)
        ds1.save_to_disk(os.path.join(self.config.root_dir, "transformed_dataset"))


In [36]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2025-12-05 16:41:13,320: INFO: common: yaml file config\config.yaml loaded successfully:]
[2025-12-05 16:41:13,322: INFO: common: yaml file params.yaml loaded successfully:]
[2025-12-05 16:41:13,324: INFO: common: created directory atartifacts:]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]g:\PROJECT\Text-Summarizer\venv\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 75400.32 examples/s]
